In [ ]:
from tensorflow.keras.backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)sess = tf.Session(config=config)set_session(sess)
sess = tf.Session(config=config)
set_session(sess) 

In [ ]:
!pip install dlib

In [ ]:
video_count=4
import cv2
import os 
import dlib
import numpy as np
p = "../mmod_human_face_detector.dat"
cnn_face_detector = dlib.cnn_face_detection_model_v1(p)

In [ ]:
path= "../Testing_Code/manoj_tiwary"
t=os.listdir(path)
#t=t[6:]
print(t[0])


In [ ]:
for i in t:
        #print(type(i))
    try:
        ##framing starts here.save the frames
        vidcap = cv2.VideoCapture(path+i)
        success,image = vidcap.read()
        count = 0
        success = True
        os.mkdir('../Test_bezos/fake_video'+str(video_count))
        frames_path="../Test_bezos/fake_video"+str(video_count)+'/frames/'
        os.mkdir(frames_path)
        print("NOW FRAMING")
        while success:
            cv2.imwrite(frames_path+'{:04d}.jpg'.format(count), image)     # save frame as JPEG file
            success,image = vidcap.read()
            print ('Read a new frame: ', success)
            count += 1
            print(count)
        name=os.listdir(frames_path)
        name.sort()
        print(name)
        k=0
        prediction_folder_path="../Test_bezos/fake_video"+str(video_count)+'/predictions/'
        os.mkdir(prediction_folder_path)  
        real_count=0
        fake_count=0
        print("NOW DETECTING")
        m=0
        #Perform face detection
        for t in name:
            #try:
            image=cv2.imread(frames_path+t)
            if image is None:
                print("Could not read input image")
                continue

            faces_cnn = cnn_face_detector(image, 1)
            if(len(faces_cnn)==0):
                print(m)
                continue
            else:
                for face in faces_cnn:

                    x = face.rect.left()
                    y = face.rect.top()
                    w= face.rect.right() - x
                    h= face.rect.bottom() - y
                    break
                y1 =y-50
                y2= y+h+30
                x1=x-15
                x2=x+w+50

                image1=image[y1:y2, x1:x2]
                #print(image1)
                #plt1.imshow(image1)
                image1 = cv2.resize(image1,(299,299))
                test_image=image1
                test_image=test_image*(1./255)
                test_image = np.expand_dims(test_image, axis = 0)
                #Testing 
                result = model.predict(test_image)
                print(result)
                #printing Fake or real on eac frame
                if(result[0][0]>=0.4):
                    print("hi1")
                    cv2.rectangle(image, (x1,y1), (x2,y2), (0,255,0), 2)
                    cv2.putText(image, 'REAL:'+str(result), (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255, 0), lineType=cv2.LINE_AA) 
                    real_count+=1
                    cv2.putText(image, 'REAL:'+str(real_count), (100, 500), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255, 0), lineType=cv2.LINE_AA) 
                    cv2.putText(image, 'FAKE:'+str(fake_count), (100, 550), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,255), lineType=cv2.LINE_AA) 
                    cv2.imwrite(prediction_folder_path+t,image)
                else:
                    print("hi2")
                    cv2.rectangle(image, (x1,y1), (x2,y2), (0,0,255), 2)
                    cv2.putText(image, 'FAKE:'+str(1-result), (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0 ,255),lineType=cv2.LINE_AA)
                    fake_count+=1
                    cv2.putText(image, 'REAL:'+str(real_count), (100, 500), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255, 0), lineType=cv2.LINE_AA)
                    cv2.putText(image, 'FAKE:'+str(fake_count), (100, 550), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,255), lineType=cv2.LINE_AA) 
                    cv2.imwrite(prediction_folder_path+t,image)
            k+=1
            print(k)
            #except:
             #   continue
        #Video Reconstruction
        img_array = []
        l=os.listdir(prediction_folder_path)
        l.sort()
        for filename in l:
            try:    

                img = cv2.imread(prediction_folder_path+filename)
                height, width, layers = img.shape
                height, width, layers = img.shape
                size = (width,height)
                img_array.append(img)
            except:
                continue
        print("NOW RECONSTRUCTING")
        save_path='../Test_bezos/fake_video'+str(video_count)+'/'
        out = cv2.VideoWriter(save_path+'fake_video'+str(video_count)+'.avi',cv2.VideoWriter_fourcc(*'DIVX'), 24, size)

        for i in range(len(img_array)):
            out.write(img_array[i])
        out.release()
        print("VIDEO COMPLETED:"+str(video_count))
        video_count+=1
    except Exception as e:
        print(e)
        continue

